In [ ]:
# Parameters for clearing the directory
hostname = 'ec2-13-60-15-218.eu-north-1.compute.amazonaws.com'
port = 22
username = 'ec2-user'
key_file = r'C:\Users\Miha\efedgauss.pem'
directory_to_clear = '/home/ec2-user'


In [ ]:
#jupyter nbconvert --to script credit_fraud_experiment.ipynb   

In [ ]:
import tarfile
import os

def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        for root, dirs, files in os.walk(source_dir):
            # Filter out directories starting with '.' and '__pycache__'
            dirs[:] = [d for d in dirs if not d.startswith('.') and d != '__pycache__']
            for file in files:
                # Filter out files starting with '.' and '.pyc' files
                if not file.startswith('.') and not file.endswith('.pyc'):
                    file_path = os.path.join(root, file)
                    tar.add(file_path, arcname=os.path.relpath(file_path, start=source_dir))

source_folder = r'C:\Users\Miha\OneDrive - Univerza v Ljubljani\Doktorski_studij\Delo\eGAUSSp_Python'
output_tarball = 'eFedGauss.tar.gz'

make_tarfile(output_tarball, source_folder)


In [ ]:
import os

print("Current working directory:", os.getcwd())
print("Does the file exist?", os.path.exists(output_tarball))


In [ ]:
def clear_remote_directory(hostname, port, username, key_file, directory_to_clear):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(hostname, port=port, username=username, key_filename=key_file)

    # Command to clear the contents of the directory
    command = f"rm -rf {directory_to_clear}/*"

    stdin, stdout, stderr = ssh.exec_command(command)
    print(stdout.read().decode())
    err = stderr.read().decode()
    if err:
        print("Error:", err)

    ssh.close()

In [ ]:
# Execute the command on the server
#clear_remote_directory(hostname, port, username, key_file, directory_to_clear)

In [ ]:
import paramiko
import os
import sys

def print_progress(transferred, total):
    progress_percentage = (transferred / total) * 100
    sys.stdout.write(f"Transfer Progress: {progress_percentage:.2f}%\r")
    sys.stdout.flush()

def transfer_file(local_path, remote_path, hostname, port, username, key_file):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(hostname, port=port, username=username, key_filename=key_file)
    
    sftp = ssh.open_sftp()
    sftp.put(local_path, remote_path, callback=print_progress)
    sftp.close()
    ssh.close()

remote_path = '/home/ec2-user/eFedGauss.tar.gz'
output_tarball = 'eFedGauss.tar.gz'  # Assuming this is the name of your local tarball

transfer_file(output_tarball, remote_path, hostname, port, username, key_file)

In [ ]:
def execute_commands_on_server(hostname, port, username, key_file, remote_path):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(hostname, port=port, username=username, key_filename=key_file)

    # Correct directory path construction
    extraction_directory = '/home/ec2-user/eFedGauss'

    commands = [
        f"mkdir -p {extraction_directory}",  # Create the directory if it doesn't exist
        f"tar -xvzf {remote_path} -C {extraction_directory}",  # Extract into the eFedGauss directory
    ]

    for command in commands:
        stdin, stdout, stderr = ssh.exec_command(command)
        print(stdout.read().decode())  # Optionally print output of each command
        err = stderr.read().decode()
        if err:
            print("Error:", err)

    ssh.close()

# Parameters for execution
python_script = 'credit_fraud_experiment.py'

# Execute commands on the server
execute_commands_on_server(hostname, port, username, key_file, remote_path)


wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
chmod +x Miniconda3-latest-Linux-x86_64.sh
./Miniconda3-latest-Linux-x86_64.sh
conda init
source ~/.bashrc

mkdir ~/tempdir
export TMPDIR=~/tempdir
cd eFedGauss
pip install -r requirements.txt